## Show the UC present on the Workspace

In [0]:
%sql
show catalogs

## Creating a Vol ume to Store the CSV File

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS testnimidbx.my_schema;

CREATE VOLUME IF NOT EXISTS testnimidbx.my_schema.my_volume

> ### After this , go to the Volumes under Catalog and upload your CSV File 

## Selecting from a CSV File


In [0]:
%sql
SELECT * FROM csv.`dbfs:/Volumes/testnimidbx/my_schema/my_volume/Customers2.csv`

## Create a Table as a select from CSV File

In [0]:
%sql
CREATE TABLE Customer_test
AS SELECT * FROM csv.`dbfs:/Volumes/testnimidbx/my_schema/my_volume/Customers2.csv`;
    
SELECT * FROM Customer_test;



## Creation of Bronze Table with the CSV File ingestion

In [0]:
csv_file_path = '/Volumes/testnimidbx/my_schema/my_volume/Customers2.csv'
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(csv_file_path)


df.createOrReplaceTempView("bronze_events")

df.display()

df.write.saveAsTable("bronze_table")


## Creation of Silver Table by applying group by on Bronze table

In [0]:
# Step 1: Read the source table
bronze_df = spark.table("bronze_table")

# Step 2: Select only the desired columns
silver_df = bronze_df.select("ProductName", "ProductColor" ,"CustomerName").groupBy("ProductName","ProductColor").count()  # Replace with your actual column names

silver_df.display()


In [0]:
# Step 3: Save the selected data into a new table
silver_df.write.saveAsTable("silver_table")


## Creation of Gold Tables ready for reporting based by applying few filters on Silver tables

In [0]:
silver_table_df = spark.table("silver_table")

gold_productcountmetrics_df = silver_table_df.groupBy("ProductName").agg({"count": "sum"}).withColumnRenamed("sum(count)", "total_count")

gold_productcountmetrics_df.display()

gold_productcountmetrics_df.write.saveAsTable("gold_count_metrics")


In [0]:
gold_productNameGT1_df = silver_table_df.select("ProductName").filter("count > 1")

gold_productNameGT1_df.display()

gold_productNameGT1_df.write.saveAsTable("gold_productNameGT1")
